In [0]:
# -*- coding: utf-8 -*-

# 階層型ニューラルネットワーク

In [0]:
# chainer のインストール
!curl https://colab.chainer.org/install | sh -

# 他のパッケージのインストール
!pip install h5py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   2797      0 --:--:-- --:--:-- --:--:--  2797
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


In [0]:
# drive のマウント（認証が必要）
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
import numpy as np
import chainer
from chainer import cuda
from chainer import Function
from chainer import report
from chainer import training
from chainer import utils
from chainer import Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList, cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [0]:
# chainerのバージョン
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
chainer.print_runtime_info()

GPU availability: True
cuDNN availablility: True
Platform: Linux-4.14.65+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.0.0
NumPy: 1.14.6
CuPy:
  CuPy Version          : 5.0.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 9020
  CUDA Driver Version   : 9020
  CUDA Runtime Version  : 9020
  cuDNN Build Version   : 7201
  cuDNN Version         : 7201
  NCCL Build Version    : 2213
iDeep: 2.0.0.post3


In [0]:
# CPU，GPUの確認
!cat /proc/cpuinfo
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
mode

In [0]:
# データ
DATA = 100
class_num = 3
data = np.zeros( (DATA,5), dtype=np.float32 )
teach = np.zeros( DATA, dtype=np.int32 )

In [0]:
# データの読み込み
def Read_data():
  count = 0
  for line in open('/content/gdrive/My Drive/data/data-1.csv', 'r' ):
    work = line[:-1].split(',')
    data[count] = work[0:5]
    teach[count] = work[5:].index( '1' )
    count+=1

In [0]:
# ネットワークの設定
class MLP(Chain):
    def __init__(self):
        super(MLP, self).__init__(
            # ネットワークの構造の設定
            l1=L.Linear(5,128),    #  入力層のニューロン数5個，中間層1のニューロン数は128個
            l2=L.Linear(128,128),  #  中間層1のニューロン数128個，中間層2のニューロン数は128個
            l3=L.Linear(128,3),    #  中間層2のニューロン数128個，出力層のニューロン数は3個
        )
    
    # 損失関数
    def __call__(self,x,y):
        # ソフトマックスクロスエントロピー誤差
        return F.softmax_cross_entropy(self.fwd(x), y)

    # 順伝播
    def fwd(self,x):
         h1 = F.relu(self.l1(x))
         h2 = F.relu(self.l2(h1))
         h3 = self.l3(h2)
         return h3

In [0]:
# 学習
def Train():
    # # 更新方法はAdam
    optimizer = optimizers.Adam() 
    optimizer.setup(model)

    # バッチ数，エポック数の設定
    n_epoch = 500
    batch_size = 10
    n_train = DATA

    # 学習
    for epoch in range(n_epoch):
        error = 0.0
        for i in range(0, n_train, batch_size):
            x = np.zeros((batch_size,5),dtype=np.float32)
            y = np.zeros( batch_size ,dtype=np.int32)

            # バッチの作成
            for j in range(batch_size):
                rnd = np.random.randint(DATA)
                x[j,:]=data[rnd,:]
                y[j]=teach[rnd]
            
            # 入力データ
            xt = Variable(cuda.to_gpu(x))
            yt = Variable(cuda.to_gpu(y))

            # 勾配の初期化→誤差の計算→逆伝播→パラメータの更新
            model.zerograds()
            loss = model(xt,yt)
            error += loss.data
            loss.backward()
            optimizer.update()

        print( epoch , ":" , error )
        
    # パラメータの保存
    serializers.save_hdf5( "/content/gdrive/My Drive/data/model.h5" , model )

In [0]:
# 予測
def Predict():
    
    # パラメータのロード
    serializers.load_hdf5( "/content/gdrive/My Drive/data/model.h5" , model )
    
    # 混合行列
    result = np.zeros( (class_num,class_num) , dtype=np.int )
    
    x = np.zeros((1,5),dtype=np.float32)
    y = np.zeros( 1 ,dtype=np.int32)
    for i in range(DATA):
      # 入力データ
      x[0,:]=data[i,:]
      y[0]=teach[i]
      
      xt = Variable(cuda.to_gpu(x))
      yt = Variable(cuda.to_gpu(y))
      
      # 予測
      predict=model.fwd(xt)
      
      # 結果の出力
      ans = np.argmax( predict.data[0] )
      print( xt.data[0] , ":" , predict.data[0] , "->" , ans , "[" , yt.data[0] , "]" )
      
      result[int(yt.data[0])][int(ans)] +=1
    
    print( "\n [混合行列]" )
    print( result )
    print( "\n 正解数 ->" ,  np.trace(result) )

In [0]:
# モデルの設定(GPU)
gpu_device = 0
cuda.get_device(gpu_device).use()
model = MLP()
model.to_gpu()


In [0]:
# データの読み込み
Read_data()

# 学習
Train()

0 : 10.601636
1 : 10.915033
2 : 10.213722
3 : 10.47108
4 : 9.987929
5 : 10.244693
6 : 9.9126835
7 : 9.706641
8 : 9.230431
9 : 9.8403015
10 : 9.763045
11 : 10.072889
12 : 9.24395
13 : 9.2970085
14 : 9.197202
15 : 9.278284
16 : 9.52419
17 : 8.291982
18 : 8.91256
19 : 9.262041
20 : 8.936502
21 : 8.044091
22 : 8.454798
23 : 8.754911
24 : 8.384319
25 : 8.856018
26 : 8.240299
27 : 8.846294
28 : 8.326254
29 : 8.048454
30 : 8.760872
31 : 8.000186
32 : 7.9900393
33 : 8.156848
34 : 8.276893
35 : 8.3602705
36 : 7.615012
37 : 8.386347
38 : 8.116522
39 : 7.452444
40 : 7.871027
41 : 7.3272433
42 : 7.840331
43 : 8.247373
44 : 7.405366
45 : 6.597603
46 : 7.5836473
47 : 7.1270604
48 : 6.4615135
49 : 6.547323
50 : 6.869278
51 : 7.3969765
52 : 6.1872005
53 : 6.520607
54 : 6.588947
55 : 7.263829
56 : 7.0080566
57 : 6.407781
58 : 6.131549
59 : 5.8698664
60 : 6.1772223
61 : 6.584764
62 : 6.2140803
63 : 5.894271
64 : 5.4518857
65 : 6.076906
66 : 6.6894445
67 : 6.740244
68 : 5.8800936
69 : 6.348275
70 : 5.454

In [0]:
# データの読み込み
Read_data()

# 予測
Predict()

[0.5 0.4 0.7 0.2 0.2] : [ 5.45772   -8.595089  -2.8256721] -> 0 [ 0 ]
[0.9 0.7 0.2 0.4 0.7] : [  4.907436   -13.943149    -0.78116745] -> 0 [ 0 ]
[0.1 0.2 0.8 0.7 0.4] : [ -3.3244624 -11.993206    3.7659817] -> 2 [ 2 ]
[0.6 0.6 0.6 0.9 0.5] : [ 0.04282355  6.3125267  -9.895386  ] -> 1 [ 1 ]
[0.2 0.3 0.4 0.8 0.9] : [-2.5609183  6.054931  -7.2395973] -> 1 [ 1 ]
[0.2 0.5 0.3 0.1 0. ] : [-2.6396732 -7.4440503  6.2207785] -> 2 [ 2 ]
[0.3 0.2 0.1 0.5 0.4] : [-2.5712442  3.1161613 -1.7805336] -> 1 [ 1 ]
[0.2 0.1 0.4 0.4 0.8] : [ 3.0259995  -8.64943    -0.56584454] -> 0 [ 0 ]
[0.3 0.6 0.9 0.4 0.7] : [-5.9152875 -0.3824377  3.4539895] -> 2 [ 2 ]
[0.8 0.2 0.1 0.7 0. ] : [  2.4647474 -25.115135   11.363614 ] -> 2 [ 2 ]
[0.  0.5 0.9 0.  0.6] : [ 4.4978385 -1.4807894 -9.938923 ] -> 0 [ 0 ]
[0.9 0.9 0.  0.9 0.4] : [  6.623454  -14.668309   -3.5695693] -> 0 [ 0 ]
[0.8 0.6 0.8 0.  0.8] : [-6.64983   -5.9365907  7.3699837] -> 2 [ 2 ]
[0.1 0.1 0.6 0.8 0.7] : [-8.110175  -4.161464   2.4140465] -> 2 [ 2 ]

In [0]:
!nvidia-smi

Sat Dec 29 04:46:28 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    72W / 149W |    156MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  